In [ ]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init stereo aruco detector scene builder

In [ ]:
from pkg.detector.aruco.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")

In [ ]:
from pkg.planning.filtering.reach_filter import ReachTrainer
from pkg.controller.combined_robot import *

In [ ]:
rtrain = ReachTrainer(scene_builder=s_builder)

## collect and train

In [ ]:
rtrain.collect_and_learn(RobotType.indy7gripper, "indy0_tcp", 
                         TRAIN_COUNT=500, TEST_COUNT=200, save_data=True, save_model=True, timeout=0.5)
import matplotlib.pyplot as plt
plt.plot(rtrain.time_plan, '.')

In [ ]:
rtrain.collect_and_learn(RobotType.panda, "panda0_hand", 
                         TRAIN_COUNT=20000, TEST_COUNT=5000, save_data=True, save_model=True, timeout=1)
import matplotlib.pyplot as plt
plt.plot(rtrain.time_plan, '.')

## load and train

In [ ]:
C_svm_list = [64, 128, 256, 512, 1024, 2048, 4096]
test_indy_list = []
# test_panda_list = []
for C_svm in C_svm_list:
    test_indy = rtrain.load_and_learn(RobotType.indy7, C_svm=C_svm)
#     test_panda = rtrain.load_and_learn(RobotType.panda, C_svm=C_svm)
    test_indy_list.append(test_indy)
#     test_panda_list.append(test_panda)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(C_svm_list, test_indy_list)
# plt.plot(C_svm_list, test_panda_list)

In [ ]:
rtrain.load_and_learn(RobotType.indy7, C_svm=1000)
# rtrain.load_and_learn(RobotType.panda, C_svm=1000)

## load and test

In [ ]:
rtrain.load_and_test(RobotType.indy7gripper)

In [ ]:
rtrain.load_and_test(RobotType.panda)

## Update label

In [ ]:
from pkg.planning.filtering.reach_filter import *

In [ ]:
ROBOT_TYPE = RobotType.indy7
TIP_LINK = "indy0_tcp"
TIMEOUT = 0.3
DATA_DIVS = ["train", "test"]
UPDATE_LABELS = [True] # to reduce feasible ones, as joint limit is reduced

In [ ]:
time_lists = []
for DATA_DIV in DATA_DIVS:
    rtrain.update_label(robot_type=ROBOT_TYPE, tip_link=TIP_LINK, data_div=DATA_DIV, update_labels=UPDATE_LABELS, timeout=TIMEOUT)
    time_lists.append(deepcopy(rtrain.time_list))
import matplotlib.pyplot as plt
plt.subplot(1,2,1)
plt.plot(sorted(time_lists[0]), '.')
plt.subplot(1,2,2)
plt.plot(sorted(time_lists[0]), '.')

## Load and visualize

In [ ]:
ROBOT_TYPE = RobotType.indy7
TIP_LINK = "indy0_tcp"
TIMEOUT = 0.3
DATA_DIV = "test"
UPDATE_LABELS = [True] # to reduce feasible ones, as joint limit is reduced

In [ ]:
rtrain.load_and_visualize(robot_type=ROBOT_TYPE, tip_link=TIP_LINK, data_div=DATA_DIV, timeout=TIMEOUT)